# Convert 3DSSG's structure to GraphViz structure

## Load Files

In [1]:
import json

In [2]:
# Paths (-> argparse로 수정)
path_3dssg_objects = "/Users/jjuke/Desktop/RVI/Projects/3DSSG/objects.json"
path_3dssg_relationships = "/Users/jjuke/Desktop/RVI/Projects/3DSSG/relationships.json"
dir_to_save = "."

In [3]:
with open(path_3dssg_objects, "r") as ro:
    data_obj = json.load(ro)
with open(path_3dssg_relationships, "r") as rr:
    data_rel = json.load(rr)

print(type(data_obj))
obj_pretty = json.dumps(data_obj, indent=4)
rel_pretty = json.dumps(data_rel, indent=4)
print(len(data_obj['scans'][0]['objects']))

<class 'dict'>
40


## Exploring Data

In [120]:
# show keys (with hierarchy)
print('============================================================')
print('For "objects.json"')
print('============================================================')
print('data is constructed by: {keys}'.format(keys=data_obj.keys()))
print()
print('"scans" is constructed by: ', data_obj['scans'][0].keys())
print()
print('"objects" is constructed by: ', data_obj['scans'][0]['objects'][0].keys())
print('"label" in "objects" example: ', data_obj['scans'][0]['objects'][0]['label'])
print()
print('"id(attribute와 relationship 관련 id - scene 내에서의 id)" example: ', data_obj['scans'][0]['objects'][0]['id']) # instance id
print('"gloal_id(label에 따른 id)" example: ', data_obj['scans'][0]['objects'][0]['global_id']) # global instance id (among classes)
print('"attributes" example: ', data_obj['scans'][0]['objects'][0]['attributes'])
print('\n\n')
print('============================================================')
print('For "relationships.json"')
print('============================================================')
print('data is constructed by: ', data_rel.keys())
print()
print('"scans" is constructed by: ', data_rel['scans'][0].keys())
print()
print('"relationships" example: ', data_rel['scans'][0]['relationships'][0]) # 0: object, 1: object, 2: relationships id, 3: semantic relationship 

For "objects.json"
data is constructed by: dict_keys(['scans'])

"scans" is constructed by:  dict_keys(['scan', 'objects'])

"objects" is constructed by:  dict_keys(['ply_color', 'nyu40', 'eigen13', 'label', 'rio27', 'affordances', 'id', 'global_id', 'attributes'])
"label" in "objects" example:  floor

"id(attribute와 relationship 관련 id)" example:  1
"gloal_id(label에 따른 id)" example:  188
"attributes" example:  {'state': ['clean'], 'shape': ['flat'], 'lexical': ['inside', 'lower', 'horizontal'], 'color': ['beige']}



For "relationships.json"
data is constructed by:  dict_keys(['scans'])

"scans" is constructed by:  dict_keys(['relationships', 'scan'])

"relationships" example:  [2, 1, 15, 'standing on']


In [4]:
print('num of data_obj: ', len(data_obj['scans']))
print('num of data_rel: ', len(data_rel['scans']))

count = 0
for i in range(len(data_obj['scans'])):
    for j in range(len(data_rel['scans'])):
        if data_obj['scans'][i]['scan'] == data_rel['scans'][j]['scan']:
            count += 1
print('num of scans: ', count)

num of data_obj:  1482
num of data_rel:  1335
num of scans:  1335


## Create New Dictionaries

* 일단은 attributes 중 state만 사용하여 visualization -> attributes 개수가 많이 줄어듦

In [5]:
# convert to the structure for vizualization
objects = []
attributes = []
relationships = []

# at first, for the first scan
scan = data_obj['scans'][0]
obj_labels = []
obj_ids = []
obj_glob_ids = []
obj_atts = []
new_ids = []

for i in range(len(scan['objects'])):
    # for objects in scene (id)
    # if scan['objects'][i]['id'] not in obj_ids:
    obj_labels.append(scan['objects'][i]['label'])
    obj_ids.append(scan['objects'][i]['id'])
    obj_glob_ids.append(scan['objects'][i]['global_id'])
    objects.append({
        "name" : scan['objects'][i]['label']
    })
    
    # for attributes
    for attribute_list in scan['objects'][i]['attributes'].values(): # dict
        for attribute in attribute_list:
            attributes.append({
                "attribute" : attribute,
                "object" : i
            })
    new_ids.append(i)

# converting id with dictionary (key: prev_id(str), value: new_id(int))
id_dict = dict(zip(obj_ids, new_ids))

# print('============================================================')
# print('============================================================')
# print('# Debug Process')
# for i in range(20):
#     print('object label : {label}  ///  object id : {id}  ///  object global id : {global_id}'
#           .format(label=obj_labels[i], id=obj_ids[i], global_id=obj_glob_ids[i]))
# print('num of objects : {num_obj}'.format(num_obj=len(obj_ids)))
# print('============================================================')
# print('============================================================')

print('num of objects: ', len(objects))
print('num of attributes: ', len(attributes))
print(obj_ids)
print(new_ids)
print(id_dict)

# for relationships
for i in range(len(data_rel['scans'])):
    scan_rel = data_rel['scans'][i]
    if scan_rel['scan'] == scan['scan']:
        for relationship in scan_rel['relationships']:
            relationships.append({
                "predicate" : relationship[3], 
                "subject" : id_dict[str(relationship[0])], # uncertain
                "object" : id_dict[str(relationship[1])] # uncertain
            })

print('num of relationships: ', len(relationships))

num of objects:  40
num of attributes:  36
['1', '450', '3', '4', '5', '6', '7', '9', '11', '12', '2', '15', '16', '17', '18', '19', '20', '21', '23', '24', '26', '220', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '28', '42', '43', '44', '46', '47', '48', '13']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
{'1': 0, '450': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '9': 7, '11': 8, '12': 9, '2': 10, '15': 11, '16': 12, '17': 13, '18': 14, '19': 15, '20': 16, '21': 17, '23': 18, '24': 19, '26': 20, '220': 21, '31': 22, '32': 23, '33': 24, '34': 25, '35': 26, '36': 27, '37': 28, '38': 29, '39': 30, '40': 31, '28': 32, '42': 33, '43': 34, '44': 35, '46': 36, '47': 37, '48': 38, '13': 39}
num of relationships:  350


## Create New Json file

In [6]:
# at first, 1st scan only
scan = str(0)
new_dict = {
    "url" : "https://www.google.com/imgres?imgurl=https%3A%2F%2Fplay-lh.googleusercontent.com%2FaFWiT2lTa9CYBpyPjfgfNHd0r5puwKRGj2rHpdPTNrz2N9LXgN_MbLjePd1OTc0E8Rl1&imgrefurl=https%3A%2F%2Fplay.google.com%2Fstore%2Fapps%2Fdetails%2FGoogle%3Fid%3Dcom.google.android.googlequicksearchbox%26hl%3Dko%26gl%3DUS&tbnid=PRtenhDnrVrfOM&vet=12ahUKEwjHoNT_85P6AhVuy4sBHQwCCtgQMygAegUIARC-AQ..i&docid=GleFcIK-j0aT3M&w=512&h=512&q=google%20image&ved=2ahUKEwjHoNT_85P6AhVuy4sBHQwCCtgQMygAegUIARC-AQ",
    "objects" : objects,
    "attributes" : attributes,
    "relationships" : relationships
}

# exploring
ind = 3
print('{index}th index of new data: \n{index}th object: {object} \n{index}th attribute: {attribute} \n{index}th relationship: {relationship}'
      .format(index=ind, object=new_dict["objects"][ind], attribute=new_dict["attributes"][ind], relationship=new_dict["relationships"][ind]))

path_to_save = dir_to_save + "/3dssg_for_viz.json"

with open(path_to_save, "w") as f:
    json.dump(new_dict, f, indent=4)

3th index of new data: 
3th object: {'name': 'trash can'} 
3th attribute: {'attribute': 'lower', 'object': 0} 
3th relationship: {'predicate': 'standing on', 'subject': 4, 'object': 0}


In [35]:
# Exploring the data with relationship
def show_relationship(index):
    print('"{subject}" is "{predicate}" the "{object}"'
        .format(subject= new_dict["objects"][int(new_dict["relationships"][index]["subject"])]["name"],
                object= new_dict["objects"][int(new_dict["relationships"][index]["object"])]["name"],
                predicate= new_dict["relationships"][index]["predicate"]))

for i in range(20):
    show_relationship(len(new_dict["relationships"]) - 1 - i) # same object 제대로 출력하는지 보기 위해 반대로 출력

"wall" is "same object type" the "wall"
"object" is "same object type" the "object"
"table lamp" is "same object type" the "table lamp"
"chair" is "same object type" the "chair"
"wall" is "same object type" the "wall"
"table lamp" is "same object type" the "table lamp"
"object" is "same object type" the "object"
"wall" is "same object type" the "wall"
"heater" is "same color" the "chair"
"box" is "same object type" the "box"
"object" is "same object type" the "object"
"chair" is "same object type" the "chair"
"object" is "same object type" the "object"
"heater" is "same color" the "door"
"trash can" is "same object type" the "trash can"
"object" is "same object type" the "object"
"door" is "same color" the "chair"
"object" is "same object type" the "object"
"object" is "same object type" the "object"
"kitchen cabinet" is "same object type" the "kitchen cabinet"
